In [18]:
import os
from dotenv import load_dotenv
load_dotenv()
import mysql.connector
import pandas as pd
import gspread
import json

# Access the environment variables.
mydb_pass = os.getenv('MYDB_PASS')
google_cred = os.getenv('GOOGLE_CRED')
gc = gspread.service_account_from_dict(json.loads(google_cred.replace('"', "'").replace("'", '"')))

In [19]:
host="production.cqof4esbua2o.us-west-2.rds.amazonaws.com"
user="leadershipDashboard"
password=mydb_pass
database="ckprodv2"
port=3306

In [23]:
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password,
  database=database,
  port=port
)

mycursor = mydb.cursor()

In [4]:
from sqlalchemy import create_engine

def create_mysql_engine(user, password, host, port, database):
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)
    return engine


In [8]:
# Get info from google sheet
id_mapping = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NzExg0HSoyrpTSkd9A49EOB4XAWZCh00w9An5p3rKa0/edit#gid=726799529')
ft_sites_gs = pd.DataFrame(id_mapping.worksheet('ft_sites').get_all_records())
ft_sites_gs.head()

,corporate_id,school_id,client_id,hubspot_id,record_type,site_name
0,91335227,0,0,13329764218,Lone Campus,Higher Education
1,91335227,ISFC,0,0,Lone Campus,Institutional Student Finance Corporation
2,91335227,RECSD,0,0,Lone Campus,Rec - Sd Miramar
3,91335227,1659,0,0,Lone Campus,Rosalind Franklin University Of Medicine
4,91335227,1151,0,4108718674,Lone Campus,San Diego State University


In [9]:
def dataframe_to_mysql(df, table_name, engine, if_exists="fail", index=False):
    """
    Load a DataFrame into a MySQL table.
    
    Parameters:
    - df (pandas.DataFrame): DataFrame to load.
    - table_name (str): Name of the table in the MySQL database.
    - engine (sqlalchemy.engine): SQLAlchemy engine instance.
    - if_exists (str): What to do if the table already exists. Options: "fail", "replace", "append". Default: "fail".
    - index (bool): Whether to write the DataFrame's index to the table. Default: False.
    """
    df.to_sql(table_name, engine, if_exists=if_exists, index=index)

In [10]:
# Write data to MySQL
engine = create_mysql_engine(user, password, host, 3306, database)
dataframe_to_mysql(ft_sites_gs, 'ft_sites', engine, if_exists="replace")

In [13]:
query = """ALTER TABLE ft_sites
MODIFY school_id VARCHAR(255);"""
mycursor.execute(query)

In [14]:
query = """ALTER TABLE ft_sites
ADD PRIMARY KEY (school_id);
"""
mycursor.execute(query)

In [24]:
### Create Corporate Table view
query = """ CREATE OR REPLACE
ALGORITHM = UNDEFINED VIEW `ft_corp_structure` AS
select
    `ft_logos`.`corporate_id` AS `corporate_id`,
    `ft_sites`.`school_id` AS `school_id`,
    `ft_sites`.`client_id` AS `client_id`,
    `ft_sites`.`hubspot_id` AS `hubspot_id`,
    `ft_logos`.`corporate_client` AS `corporate_client`,
    `ft_sites`.`site_name` AS `site_name`
from
    (`ft_sites`
left join `ft_logos` on
    ((`ft_sites`.`corporate_id` = `ft_logos`.`corporate_id`)));
"""
mycursor.execute(query)

In [27]:
def pull_mysql_table(table_name):
    query = f"SELECT * FROM {table_name}"

    mycursor.execute(query)
    raw_data = mycursor.fetchall()

    field_names = [i[0] for i in mycursor.description]
    raw_df = pd.DataFrame(raw_data, columns=field_names)

    return raw_df

In [28]:
school_df = pull_mysql_table('school')

In [29]:
school_df

,schoolID,name,acronym,state,isCareer,timezone,textAimSchoolID,hasCortex,hasComplianceReports,createdAt,updatedAt
0,1,Location One,None,CA,1,America/Los_Angeles,None,0,0,2019-01-04 23:37:11,2019-01-04 23:37:11
1,10010,American Samoa Community College,None,AS,0,None,None,0,0,2019-01-04 23:37:11,2019-01-04 23:37:11
2,10014,Garrett Community College,None,MD,0,None,None,0,0,2019-01-04 23:37:11,2019-01-04 23:37:11
3,10015,Bayamon Central University,None,PR,0,None,None,0,0,2019-01-04 23:37:11,2019-01-04 23:37:11
4,10017,Payne TheologiCalifornia Seminary,None,OH,0,America/New_York,None,0,0,2019-01-04 23:37:11,2019-01-04 23:37:11
...,...,...,...,...,...,...,...,...,...,...,...
7012,VIS,Milan - Visalia,None,CA,1,America/Los_Angeles,None,0,0,2020-10-30 15:47:05,2020-10-30 15:47:05
7013,WASLA,Charter College - Wasilla,None,AK,1,America/Anchorage,None,0,0,2022-11-16 01:59:06,2022-11-16 01:59:06
7014,WCCC,Washington County Career Center Adult Tech,None,NY,1,America/New_York,None,0,0,2022-03-08 22:11:10,2022-03-08 22:11:10
7015,WENCH,Charter College - East Wenatchee,None,WA,1,America/Los_Angeles,None,0,0,2022-11-16 01:58:58,2022-11-16 01:58:58
